In [3]:
import sys
import json
import os.path
from os import path
import numpy as np
import tensorflow as tf
from google.colab import drive
from keras import backend as K
from keras.models import Model
from keras.layers import Input, Dense, Lambda
from keras.initializers import RandomUniform, Constant
from keras.optimizers import RMSprop
from keras.constraints import unit_norm
from keras import regularizers
from keras.callbacks import Callback
from keras.losses import mean_squared_error
from tensorflow.python.framework.ops import disable_eager_execution

disable_eager_execution()

drive.mount('/content/gdrive')

device_name = tf.test.gpu_device_name()
#if device_name != '/device:GPU:0':
#  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Found GPU at: 


In [4]:
def data_prep(system_name, number_trajs, predictive_step, input_dir, whiten_data, num_ops_used):
    """ 
    Read the input trajectory files. Prepare X, X_dt trajectory and corresponding re-weighting factors. Whiten if requested.
        
        Parameters
        ----------
        system_name : string
            Name of the system.
            
        number_trajs : int
            Number of trajectories.
        
        predictive_step : int
            Predictive time delay (in number of samples).

        input_dir : str
            Input dir where .npy files are saved
          
        whiten_data : bool 
            Whether to whiten the data or not (setting mean 0 and variance 1)

        Returns
        -------
        X : np.array
            present trajectory.
         
        X_dt : np.array
            future trajectory.
            
        W1 : np.array
            re-weighting factors in objective function before P(X_t | \chi )
            
        W2 : np.array
            re-weighting factors in objective function before P(X | \chi )
    """
    
    # For each trajectory
    for j in range(number_trajs):

        # trajectory npy file names
        traj_file_name = input_dir + 'x_'+system_name+'_%i.npy'%j   # present trajectory of the shape n*d, where n is the MD steps and d is the number of order parameters (PLUMED output)
        w_file_name    = input_dir + 'w_'+system_name+'_%i.npy'%j   # weights correspond to trajectory in "traj_file_name". Calculated by exp(beta*V)   

        # If time delay = 0
        if predictive_step==0:

            # Load trajectory data
            x = np.load(traj_file_name)
            x_dt = x[:,:]      

            # Load weights
            w1 = np.load(w_file_name)   
            w2 = np.zeros(np.shape(w1))      

        # If time delay =! 0
        else:

            # Load trajectory data
            x = np.load(traj_file_name)

            # Save data, shifted "time delay" wrt each other
            x_dt = x[predictive_step: , :]  # x_dt (advanced dt with respect to X)
            x = x[:-predictive_step, :]     # x

            # Load weights
            w = np.load(w_file_name)

            # Save corresponding weights
            w_x = w[:-predictive_step] # weights for x
            w_y = w[predictive_step:]  # weights for x_dt   

            # Will be used in the loss function 
            w1 = ( w_x * w_y )**0.5
            w2 =  w_x**0.5*( w_x**0.5 - w_y**0.5)

        # 2. For subsequent trajectories
        try:
            # Append trajectory data to full dataset
            X = np.append(X, x, axis = 0)
            X_dt = np.append(X_dt, x_dt, axis = 0)

            # Append trajectory weights to all weights
            W1 = np.append(W1, w1, axis = 0)
            W2 = np.append(W2, w2, axis = 0)

        # 1. For the first trajectory
        except:
            X = x
            X_dt = x_dt

            W1 = w1
            W2 = w2

    normalization_factor = np.sum(W1)/len(W1)  

    W1 /= normalization_factor
    W2 /= normalization_factor    

    print('length of data: %i'%np.shape(X)[0])
    print('number of order parameters: %i'%np.shape(X)[1])
    print('min re-weighting factor: %f'%np.min(W1))
    print('max re-weighting factor: %f'%np.max(W1))  

    if whiten_data:
        X, scaling_factors = scaling(X) 
        X_dt -= np.mean(X_dt, axis=0)
        X_dt /= scaling_factors
    else:
        scaling_factors = np.ones(num_ops_used) 

    return X, X_dt, W1, W2, scaling_factors

def random_pick(x, x_dt, w1, w2, training_len, validation_len):
    """ 
    Finds training set: ramdomly pick (x, x_dt) pair from data set of size "training_len"  
    Finds validation set: ramdomly pick (x, x_dt) pair from data set of size "validation_len" 

        Parameters
        ----------
        x : np.array
            present trajectory.
         
        x_dt : np.array
            future trajectory.
            
        w1 : np.array
            re-weighting factors in objective function before P(X_t | \chi )
            
        w2 : np.array
            re-weighting factors in objective function before P(X | \chi )
            
        training_len: int
            length of the return training set

        validation_len: int
            length of the return validation set  
                
        Returns
        -------

        train_x, vali_x : np.array
            randomly selected data points from trajectory for training and validation sets.
         
        train_xdt, vali_xdt : np.array
            train_x and vali_x shifted dt

        train_w1, vali_w1 : np.array
            corresponding re-weighting factors in objective function before the decoder P(X_t | \chi ) (?)
            
        train_w2, vali_w2 : np.array
            corresponding re-weighting factors in objective function before encoder P(X | \chi ) (?)

    """   
    # Find total number of samples per OP
    num_samples = np.shape(x)[0]

    # Check we have enough samples 
    if ((training_len+validation_len)>num_samples):
      print("ERROR: Chosen training and validation lengths exceeds total amount of used samples")
      sys.exit()

    # Create indices for all samples
    indices = np.arange(num_samples) 

    # Shuffle indices  
    np.random.shuffle(indices)

    # First "training_len" samples will be used for training
    train_indices = indices[:training_len]

    # Next "validation_len" samples will be used for validation
    val_indices = indices[training_len:training_len+validation_len]

    # Find samples for training set, all columns (all OPs)
    train_x = x[train_indices, :]
    train_xdt = x_dt[train_indices, :]

    # Find samples for validation set, all columns (all OPs)
    vali_x = x[val_indices, :]
    vali_xdt = x_dt[val_indices, :]

    # Find weights for training set, all columns (all OPs)
    train_w1 = w1[train_indices]
    train_w2 = w2[train_indices]

    # Find weights for validation set, all columns (all OPs)
    vali_w1 = w1[val_indices]
    vali_w2 = w2[val_indices]

    print('{} data points are used in this training'.format(training_len))
    
    return train_x, train_xdt, train_w1, train_w2, vali_x, vali_xdt, vali_w1, vali_w2
    
def scaling(x):
    """ make order parameters with mean 0 and variance 1
        return new order parameter and scaling factors
        
        Parameters
        ----------
        x : np.array
            order parameters
            
        Returns
        ----------
        x : np.array
            order parameters after rescaling
        
        std_x : np.array
            rescaling factors of each OPs
              
     """ 
   
    x = x-np.mean(x, axis =0)
    std_x = np.std(x, axis =0)

    return x/std_x, std_x

def sampling(args):

	"""Sample the latent variable
	from a Normal distribution."""

	s_mean= args
	epsilon = K.random_normal(shape=(batch_size,rc_dim), mean=0.0, stddev=s_vari )
	s_noise = s_mean +  epsilon

	return s_noise

def dynamic_correction_loss(x, w1, w2):
    """Custom loss function with dynamic correction
       
       Parameters:
       -----------
       x: input data
       w1:
       w2:
       """

    def custom_loss(y_true, y_pred ):
         ce1 = mean_squared_error(y_true, y_pred )
         ce2 = mean_squared_error(x, y_pred)  
         return (w1[:,0]*ce1+w2[:,0]*ce2) 

    return custom_loss

class WeightsHistory(Callback):

    def on_train_begin(self, logs={}):
        self.losses = []
        self.losses_vali = []
        self.weights0 = []

    def on_epoch_end(self, epoch, logs={}):
        self.losses.append(logs.get('loss'))
        self.losses_vali.append(logs.get('val_loss'))
        self.weights0.append( prave.layers[1].get_weights()) # Coefficients of first layer

def COLVAR2npy(Name, Temperature, num_OPs_used, Dir ='input/', first_row = 0, Bias = False, n_bias_convert = 1):
    ''' 
    Convert the COLVAR (file that contains the trajectory and corresponding biasing potential) to npy file.
    Here we assume that the COLVAR is of the shape: n rows * (1+d1+d2+d3+d4) columns, where:
        
        - n is the number of rows of data printed by plumed (depends on the total simulation time, time step and plumed stride)
        
        - the first column is the simulation time in ps
        - d1 is the number of order parameters that will be used to train the TAE
        - d2 is the number of order parameters that won't be used to train the TAE 
        - d3 is the number of reaction coordinates
        - d4 is the number of bias potentials that are added during the simulation

        where d3 might be different from d4 as we might have a 2D RC with Vbias(RCx, RCy)
        
        Parameters
        ----------
        Name : string
            Name of the system.
            
        Temperature : float
            Temperature in unit of Kelvin.
        
        num_OPs_used : int
            Dimensionality of order parameter space (feature space) used to train the TAE
        
        Dir: string
            Directory of input and output (.npy) files.
        
        first_row: int
            First row index to consider from the input dataset

        Bias: Bool
            Whether the trajectory is from a biased MD.
            When false re-weighting factors are set to 1. 
            When true, re-weighting factors are calculated and saved. 

        n_bias_convert: int
            (only used if bias = True) number of biases that will be converted into re-weighting factors, we might want to leave some
            without re-weighting to apply the "washing out" trick. Usually equal to the number of already learned RC linear components
    
        Returns
        -------
        None
        
    '''  

    # If the traj is biased 
    if Bias:

        # Files to save data (x) and weights (w)
        traj_save_name = Dir + 'x_' + Name
        bias_save_name = Dir + 'w_' + Name

        if path.exists(traj_save_name + '.npy') and path.exists(bias_save_name + '.npy'):
            print( 'npy files already exist, delete them if you want to generate new ones.')
        else:
          
            # Load input data
            Data = np.loadtxt(Dir + Name)   

            # x: time and the value of the order parameters for all frames
            x = Data[::, 1:num_OPs_used+1]  # all rows, columns of used OP

            # w has the value of total bias for every frame - associated to a certain value of the RC and order parameters. That sample in OP space will carry a weight = exp(V/KbT)
            w = np.sum( Data[::, -n_bias_convert :], axis = 1)  # All rows, summing all columns corresponding to biases we want to re-weight

            # only a part of the full trajectory will be saved
            x = x[first_row:, :]   # rows used
            w = w[first_row:]      # Total bias to re-weight at each time step / sample

            # Calculate re-weighting factor for each sample
            re_weighting_factor = np.exp(w/(0.008319*Temperature))   # Here we assume that unit of bias is kJ
            
            # Save OPs
            np.save(Dir+'x_'+ Name, x)
            
            # Save re-weighting factors - not normalized
            np.save(Dir+'w_'+ Name, re_weighting_factor) 

    # If the traj is unbiased 
    else: 

        # Files to save data (x) and weights (w)
        traj_save_name = Dir + 'x_unbiased_' + Name
        bias_save_name = Dir + 'w_unbiased_' + Name     

        if path.exists(traj_save_name+'.npy') and path.exists(bias_save_name+'.npy'):
            print( 'npy files already exit, delete them if you want to generate new ones.')
        else:

            # Load input data
            Data = np.loadtxt(Dir+ Name) 

            # x: time and the value of the order parameters for all frames
            x = Data[::, 1:num_OPs_used+1]     # all rows, columns of used OP

            # only a part of the full trajectory will be used 
            x = x[first_row:, :] 

            # trajectories are treated as unbiased - re-weighting factors = 1
            re_weighting_factor = np.ones( np.shape(x[:,0]))    

            # Save OPs
            np.save(Dir+'x_unbiased_'+ Name, x) 

            # Save re-weighting factors
            np.save(Dir+'w_unbiased_'+ Name, re_weighting_factor)
    
def save_result(system_name, op_dim, time_delay, trials, save_path='output/'):
    ''' save final result (linear combinaton coefficients of OPs) to a txt file
        Parameters
        ----------
        system_name : string
            Name of the system.
            
        op_dim : int
            Dimensionality of order parameters.
        
        time_delay : int
            Predictive time delay.
        
        trials: list
            Indexes of all trials.
        
        save_path: string
            Directory of where the final result is saved.
            
        Returns
        ----------
            None  
            Result is saved to a txt file.
    
    '''
    weights = []
    for dt in time_delay:
        Loss = []
        Weights = []
        for trial in trials: 
            save_dir = system_name+'_dt'+str(dt)+'_trial'+str(trial)+'.npy'
            Result_loss = np.load(save_path+'Loss_'+save_dir) 
            Result_weights = np.load(save_path+'Weights_'+save_dir) 
            Loss.append(np.average( Result_loss[-2:,-1] ))
            Weights.append( Result_weights[-1,:,:] )
        
        Weights = np.array( Weights )
        min_index = np.argmin(Loss)
        weights.append( Weights[min_index,:,:] )
    weights = np.array(weights)
    
    ###save weights vs. time delay###
    head = 'time_delay/MD_step  '
    number_rcs = np.shape(Weights)[-1]
    print('There are %i reaction coordinates'%number_rcs)
    for j in range(op_dim):
        head+='op%i  '%(j+1)
    for j in range(len(time_delay)):
         result_given_dt = np.concatenate((np.transpose( [[time_delay[j]]*number_rcs] ), np.transpose(weights[j,:,:])), axis =-1)
         try:
             final_result = np.concatenate((final_result, result_given_dt), axis=0)
         except:
             final_result = result_given_dt
            
    np.savetxt(save_path+'final_result_'+system_name+'.txt', final_result, header =head, newline='\n')
    

In [5]:
########################
### Global Variables ###

# Paths   
save_path = '/content/gdrive/MyDrive/WORK/NostrumBioDiscovery/2022_ENSEM/RAVE/Data/output/'     # path to the directory that saves output files
input_dir = '/content/gdrive/MyDrive/WORK/NostrumBioDiscovery/2022_ENSEM/RAVE/Data/input/'

# Simulation details
system_name = 'p53'                                       # name of the system. Input files should be named: "system_name_i" starting with i=0
n_trajs = 8                                               # number of trajectories in input folder  NOTE: automate with number of "system_name_i" files 
T = 300                                                   # Temperature in unit of Kelvin 
num_ops_used = 12                                         # number of order parameters that should be used to define the RC
first_row = 0                                             # initial row index from the data to consider for training (to exclude equilibration if any)
bias = False                                              # When false, reweigting factors are set to 1. When true, reweigting factors are calculated, saved and used
n_bias_convert = 1                                        # (only used if bias = True) number of biases that will be converted into re-weighting factors starting to count from the end column, 
                                                          # we might want to leave some without re-weighting to apply the "washing out" trick, usually equal to the number of already learned RC linear components
# Predictive time delay        
time_delay= [1, 2, 4]                                        # predictive time delay/s to use (in number of samples) - 0.5 ps between samples
    
# Network variables
training_size = 5000000                                   # "training_size" data points will be randomly picked from the whole data set and used to do the training
validation_size = 5000000                                 # "validation_size" data points will be randomly picked from (the whole data set - training set) and used to do the validation 
batch_size = 50000                                        # total number of training data point n should be a multiple of batch_size 
epochs = 2                                                # number of epochs to train the model
rc_dim = 1                                                # dimensionality of the RC
nn_dim = 128                                              # number of cells in each layer of the decoder NN
s_vari = 0.005                                            # constant variance of the gaussian noise added before the decoder
learning_rate = 0.0002                                    # constant learning rate
decay = 0.0                                               # decay of learning rate
trials = range(3)                                         # number of independent trials to train the time-lagged autoencoder
random_uniform = RandomUniform(minval=-0.5, maxval=0.5) 
set_constant = Constant(value = 0.5**0.5)
whiten_data = True                                        # If True, then normalize the order parameters (set mean 0 and variance = 1)

######################### 
### input preparation ###
for traj_index in range(n_trajs):

    input_name = system_name+'_%i'%traj_index

    # Convert each trajectory's COLVAR file to npy file
    COLVAR2npy(input_name, T, num_ops_used, input_dir, first_row, bias, n_bias_convert)

# Update for npy file name
if not bias:
    system_name = 'unbiased_' + system_name       

################################# 
### set predictive time delay ###
for dt in time_delay:      

    #####################################
    ### load the dataset, whiten data ###
    X, X_dt, W1, W2, scaling_factors = data_prep(system_name, n_trajs, dt, input_dir, whiten_data, num_ops_used)

    ############################   
    ### run different trials ###  
    for trial in trials:   

        ############################################  
        ### Variational Autoencoder architecture ###
        input_Data = Input(batch_shape=(batch_size, num_ops_used))  
        input_w1 = Input(shape=(1,))    
        input_w2 = Input(shape=(1,))  
        linear_encoder = Dense(rc_dim, activation=None, use_bias=None, kernel_regularizer=regularizers.l1(0.0), kernel_initializer='random_uniform', kernel_constraint = unit_norm(axis=0))(input_Data)
              
        s = Lambda(sampling)(linear_encoder)  # Custom layer with the gaussian sampling
        hidden_a = Dense(nn_dim, activation='elu', kernel_initializer='random_uniform')(s)
        hidden_b = Dense(nn_dim, activation='elu', kernel_initializer='random_uniform')(hidden_a)
        y_reconstruction = Dense( num_ops_used, activation=None, kernel_initializer='random_uniform')(hidden_b)
        
        ##########################################
        ### Randomly pick samples from dataset ###
        train_x, train_xdt, train_w1, train_w2, vali_x, vali_xdt, vali_w1, vali_w2 = random_pick(X, X_dt, W1, W2, training_size, validation_size)
      
        #############################################
        ### Prepare the PRAVE and train the PRVAE ###

        # Put the model together
        prave = Model([input_Data, input_w1 , input_w2] ,y_reconstruction)            
        
        # Create optimizer class
        rmsprop = RMSprop(learning_rate=learning_rate, decay = decay)
        
        # Compile the model
        prave.compile(optimizer=rmsprop,loss=dynamic_correction_loss(input_Data, input_w1, input_w2))
        
        # Create class "WeightsHistory"
        history = WeightsHistory()

        # Fit the model, save loss and validation loss on epoch end
        History = prave.fit( [train_x,train_w1,train_w2], train_xdt,
          shuffle=True,
          epochs=epochs,
          batch_size=batch_size,
            validation_data=([vali_x,vali_w1,vali_w2], vali_xdt),
            callbacks = [history])
                
        ####################
        ### Save results ###
        Loss = np.array( history.losses )
        Val_Loss = np.array( history.losses_vali )     

        # Coefficients of first layer (linear encoder)    
        Weights0 = np.array( history.weights0 )[:,0,:,:]  #  num epochs x ? x dim OP x dim latent space (num RC)
        
        # w_norm = np.linalg.norm(Weights0,  axis=1)
        for op_index in range( num_ops_used ):
            Weights0[:,op_index,:] /= scaling_factors[op_index] # rescale back to RC weights of non-standardized OPs

        for rc_index in range( rc_dim ):
            Weights0[:, :, rc_index] = np.transpose( np.transpose( Weights0[:, :, rc_index] ) / np.linalg.norm(Weights0[:, :, rc_index], axis=1)) #normalize the rc weights
            
        Loss = np.expand_dims(Loss, axis=-1)
        Val_Loss = np.expand_dims(Val_Loss, axis=-1)

        result_loss = np.concatenate((Loss, Val_Loss) , axis =-1)

        result_weights = Weights0         

        K.clear_session()

        print('!!!!')
        print(np.shape(result_weights))
        
        # Name for results - to avoid overwriting different dt and trials for a certain set of hyper-parameters
        save_info = system_name+'_dt'+str(dt)+'_trial'+str(trial) 

        # Save loss and coefficients
        np.save(save_path+'Loss_'+save_info, result_loss)
        np.save(save_path+'Weights_'+save_info, result_weights)

        del History

    del X, X_dt, W1, W2, scaling_factors

# Hyper-parameters 
hyper_param={  
    "train_size": training_size,
    "batch_size": batch_size,
    "epochs": epochs,
    "trials": trial+1,
    "layer_dimension": nn_dim,
    "learning_rate": learning_rate,
    "decay": decay,
    "decoder_noise_variance": s_vari,
    "num_dt_values": len(time_delay),
    "rc_dim": rc_dim
} 
  
# Dump hyperparameters  
json_object = json.dumps(hyper_param, indent = 4)

# Writing to sample.json
json_file = save_path + "hyper_parameters.json"
with open(json_file, "w") as outfile:
    outfile.write(json_object)

### analyze and save the results ###
save_result(system_name, num_ops_used, time_delay, trials, save_path)  


npy files already exit, delete them if you want to generate new ones.
npy files already exit, delete them if you want to generate new ones.
npy files already exit, delete them if you want to generate new ones.
npy files already exit, delete them if you want to generate new ones.
npy files already exit, delete them if you want to generate new ones.
npy files already exit, delete them if you want to generate new ones.
npy files already exit, delete them if you want to generate new ones.
npy files already exit, delete them if you want to generate new ones.
length of data: 15479992
number of order parameters: 12
min re-weighting factor: 1.000000
max re-weighting factor: 1.000000
5000000 data points are used in this training
Train on 5000000 samples, validate on 5000000 samples
Epoch 1/2
5000000/5000000 [==============================] - ETA: 0s - loss: 0.9511

/usr/local/lib/python3.9/dist-packages/keras/engine/training_v1.py:2335: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


5000000/5000000 [==============================] - 43s 9us/sample - loss: 0.9511 - val_loss: 0.9113
Epoch 2/2
5000000/5000000 [==============================] - 41s 8us/sample - loss: 0.8950 - val_loss: 0.8789
!!!!
(2, 12, 1)
5000000 data points are used in this training
Train on 5000000 samples, validate on 5000000 samples
Epoch 1/2
5000000/5000000 [==============================] - 42s 8us/sample - loss: 0.9338 - val_loss: 0.8762
Epoch 2/2
5000000/5000000 [==============================] - 42s 8us/sample - loss: 0.8600 - val_loss: 0.8433
!!!!
(2, 12, 1)
5000000 data points are used in this training
Train on 5000000 samples, validate on 5000000 samples
Epoch 1/2
5000000/5000000 [==============================] - 42s 8us/sample - loss: 0.9167 - val_loss: 0.8275
Epoch 2/2
5000000/5000000 [==============================] - 40s 8us/sample - loss: 0.8059 - val_loss: 0.7888
!!!!
(2, 12, 1)
length of data: 15479984
number of order parameters: 12
min re-weighting factor: 1.000000
max re-weigh